### Tabelas de partos para dashboard

#### Libs e importando os dados

In [3]:
GRUPO_PROCED = {
    'Normal/Cesário': ['0310010039', '0310010055', '0411010034', '0411010042'],
    'De Risco': ['0310010047', '0411010026']
}

In [4]:
import pandas as pd

df = pd.read_parquet('../../app/datasets/partos_pe.parquet.gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 83797 entries, 2622102264276 to 2622109033346
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   GESTAO     83797 non-null  int32 
 1   MUNIC_RES  83797 non-null  int32 
 2   MUNIC_MOV  83797 non-null  int32 
 3   PROC_REA   83797 non-null  object
 4   ANO_CMPT   83797 non-null  int32 
 5   MES_CMPT   83797 non-null  int32 
 6   NASC       83797 non-null  object
 7   SEXO       83797 non-null  int32 
 8   DT_INTER   83797 non-null  object
 9   DT_SAIDA   83797 non-null  object
 10  IDADE      83797 non-null  int32 
 11  MORTE      83797 non-null  object
 12  CNES       83797 non-null  object
 13  RACA_COR   83797 non-null  int32 
 14  MARCA_UTI  83797 non-null  object
 15  MARCA_UCI  83797 non-null  object
dtypes: int32(8), object(8)
memory usage: 8.3+ MB


In [5]:
df_geres = pd.read_parquet('../../app/datasets/localidade_pe.parquet.gzip')
df_geres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   codmunres  185 non-null    int32 
 1   municipio  185 non-null    object
 2   cod_geres  185 non-null    int32 
 3   geres      185 non-null    object
 4   muncoddv   185 non-null    int32 
 5   geresnome  185 non-null    object
 6   microreg   85 non-null     object
dtypes: int32(3), object(4)
memory usage: 8.1+ KB


In [13]:
df_geres.head()

,codmunres,municipio,cod_geres,geres,muncoddv,geresnome,microreg
0,260005,Abreu e Lima,2601,I GERES,2600054,I GERES - Recife,MICRO I
1,260105,Araçoiaba,2601,I GERES,2601052,I GERES - Recife,MICRO I
2,260290,Cabo de Santo Agostinho,2601,I GERES,2602902,I GERES - Recife,MICRO III
3,260345,Camaragibe,2601,I GERES,2603454,I GERES - Recife,MICRO II
4,260440,Chã de Alegria,2601,I GERES,2604403,I GERES - Recife,MICRO II


In [6]:
df_tipo_parto = pd.DataFrame.from_dict(GRUPO_PROCED, orient='index').\
    T.\
    stack().\
    reset_index().\
    rename(columns={'level_1': 'TIPO_PARTO', 0: 'PROC_REA'}).\
    drop('level_0', axis=1)

df_tipo_parto

,TIPO_PARTO,PROC_REA
0,Normal/Cesário,0310010039
1,De Risco,0310010047
2,Normal/Cesário,0310010055
3,De Risco,0411010026
4,Normal/Cesário,0411010034
5,Normal/Cesário,0411010042


In [14]:
df_partos = df.reset_index().\
    merge(df_geres[['codmunres', 'municipio', 'geresnome']], how='left', left_on='MUNIC_RES', right_on='codmunres').\
    drop('codmunres', axis=1).\
    rename(columns={'geresnome': 'GERES_RES', 'municipio': 'NM_MUNIC_RES'}).\
    merge(df_geres[['codmunres', 'geresnome']], how='left', left_on='MUNIC_MOV', right_on='codmunres').\
    drop('codmunres', axis=1).\
    rename(columns={'geresnome': 'GERES_MOV', 'municipio': 'NM_MUNIC_MOV'}).\
    merge(df_tipo_parto, how='left', on='PROC_REA')

#### Tabela Geres internação para Geres residência

In [8]:
df_partos.\
    query('GESTAO == 1').\
    groupby(['GERES_MOV', 'GERES_RES'], as_index=False)['N_AIH'].\
    count().\
    pivot_table(index='GERES_MOV', columns='GERES_RES', values='N_AIH', aggfunc='sum', fill_value=0, margins=True,margins_name='Total').reset_index()

GERES_RES,GERES_MOV,I GERES - Recife,II GERES - Limoeiro,III GERES - Palmares,IV GERES - Caruaru,IX GERES - Ouricuri,V GERES - Garanhuns,VI GERES - Arcoverde,VII GERES - Salgueiro,VIII GERES - Petrolina,X GERES - Afogados da Ingazeira,XI GERES - Serra Talhada,XII GERES - Goiana,Total
0,I GERES - Recife,15821,382,749,192,1,37,12,3,3,11,6,188,17405
1,II GERES - Limoeiro,1,618,1,39,0,0,0,0,0,0,0,1,660
2,III GERES - Palmares,2,26,839,66,0,1,3,0,0,0,1,1,939
3,IV GERES - Caruaru,0,16,9,5417,0,4,5,0,0,2,0,0,5453
4,IX GERES - Ouricuri,0,0,0,0,2359,0,0,2,1,0,0,0,2362
5,V GERES - Garanhuns,1,0,5,37,1,2493,11,0,0,0,0,0,2548
6,VI GERES - Arcoverde,0,0,0,10,0,1,1063,0,0,0,0,0,1074
7,VII GERES - Salgueiro,0,0,0,0,0,0,0,32,0,0,0,0,32
8,VIII GERES - Petrolina,0,0,0,0,0,0,0,1,990,1,0,0,992
9,X GERES - Afogados da Ingazeira,0,0,1,0,0,0,2,1,0,434,1,0,439


#### Tabela para Mapa

In [9]:
geres_selecionada = 'I GERES - Recife'

In [16]:
df_partos.\
    query("GERES_MOV == @geres_selecionada").\
    groupby(['GERES_MOV', 'MUNIC_RES', 'NM_MUNIC_RES', 'TIPO_PARTO'], as_index=False)['N_AIH'].\
    count()

,GERES_MOV,MUNIC_RES,NM_MUNIC_RES,TIPO_PARTO,N_AIH
0,I GERES - Recife,260005,Abreu e Lima,De Risco,298
1,I GERES - Recife,260005,Abreu e Lima,Normal/Cesário,353
2,I GERES - Recife,260010,Afogados da Ingazeira,De Risco,27
3,I GERES - Recife,260010,Afogados da Ingazeira,Normal/Cesário,6
4,I GERES - Recife,260030,Agrestina,De Risco,31
...,...,...,...,...,...
323,I GERES - Recife,261630,Vicência,Normal/Cesário,21
324,I GERES - Recife,261640,Vitória de Santo Antão,De Risco,268
325,I GERES - Recife,261640,Vitória de Santo Antão,Normal/Cesário,679
326,I GERES - Recife,261650,Xexéu,De Risco,25
